# Biclustering CC
Implementation proposed by Cheng & Church in Biclustering of Expression Data

In [3]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import consensus_score

In [4]:
%%latex
Define Mean Squeae Residue (MSR)$H(I,J)$:
$$H(I,J) = \frac{1}{|I||J|} \sum_{i \in I} \sum_{j \in J} (a_{ij} - a_{Ij} - a_{iJ} + a_{IJ})^2$$
where:
$$a_{iJ} = \frac{1}{|J|} \sum_{j \in J} a_{ij}$$
$$a_{Ij} = \frac{1}{|I|} \sum_{i \in I} a_{ij}$$
$$a_{IJ} = \frac{1}{|I||J|} \sum_{i \in I, j \in J} a_{ij}$$


<IPython.core.display.Latex object>

In [5]:
class MSR(object):
    def __init__(self,data):
        self.data=data
        self.n, self.m = data.shape
        self.aiJ = np.mean(data,axis=1)
        self.aIj = np.mean(data,axis=0)
        self.aIJ = np.mean(data)
        self._H = None
        self._HiJ = None
        self._HIj = None
    
    @property
    def H(self):
        if self._H is None:
            print ("computing MSR ...")
            self._H = self._compute_H()
            print ("MSR VALUE " + str(self._H))
        return self._H
        
    @property
    def HiJ(self):
        if self._HiJ is None:
            self._HiJ = self._compute_HiJ()
        return self._HiJ
    
    @property
    def HIj(self):
        if self._HIj is None:
            self._HIj = self._compute_HIj()
        return self._HIj
    
    def _compute_H(self):
        H = 0
        for i in range(self.n):
            for j in range(self.m):
                H  += (self.data[i,j] - self.aIj[j] - self.aiJ[i] + self.aIJ ) ** 2
        H *= 1.0/(self.n + self.m)       
        return H
    
    def _compute_HiJ(self):
        HiJ = np.zeros(self.n)
        for i in range(self.n):
            for j in range(self.m):
                HiJ[i] += ( self.data[i,j] - self.aIj[j] - self.aiJ[i] + self.aIJ )**2
        HiJ *= 1.0/self.m
        return HiJ


    def _compute_HIj(self):
        HIj = np.zeros(self.m)
        for j in range(self.m):
            for i in range(self.n):
                HIj[j] += ( self.data[i,j] - self.aIj[j] - self.aiJ[i] + self.aIJ )**2
        HIj *= 1.0/self.n
        return HIj


        

In [6]:
import random
data  = np.random.random((5, 5))
data

array([[ 0.7714686 ,  0.38901062,  0.50071368,  0.95953905,  0.4719957 ],
       [ 0.92451367,  0.13852356,  0.95933971,  0.25337161,  0.68681878],
       [ 0.90676724,  0.10489289,  0.98191844,  0.61737656,  0.28549578],
       [ 0.98601928,  0.20642548,  0.3342513 ,  0.79954132,  0.49948114],
       [ 0.33105844,  0.49796544,  0.54729449,  0.64151175,  0.85167327]])

In [7]:
msr = MSR(data)
print ("aIJ= " + str(msr.aIJ))
print ("H= " + str(msr.H))
print ("aiJ= " + str(msr.aiJ))
print ("aIj= " + str(msr.aIj))

aIJ= 0.585878711288
computing MSR ...
MSR VALUE 0.118874651634
H= 0.118874651634
aiJ= [ 0.61854553  0.59251347  0.57929018  0.5651437   0.57390068]
aIj= [ 0.78396545  0.2673636   0.66470352  0.65426805  0.55909293]


In [8]:
def remove_unique_nodes(data, delta, I=None, J=None):
    it = 1
    
    if I is None:
        I = np.arange(len(data))
    
    if J is None:    
        J = np.arange(len(data[0]))
        
    while True:
        it += 1
        
        msr = MSR(data[I][:,J])
        
        if msr.H <delta:
            break
            
        if len(I) == 1 or len(J) == 1:
            break
        
        row_idx_to_remove = np.argmax(msr.HiJ)
        col_idx_to_remove = np.argmax(msr.HIj)
        
        if msr.HiJ[row_idx_to_remove] > msr.HIj[col_idx_to_remove]:
            print("removing row " + str(row_idx_to_remove))
            I = np.delete(I,row_idx_to_remove)
            
        else:
            print("removing col " + str(col_idx_to_remove))
            J = np.delete(J, col_idx_to_remove)
            
    return (data[I][:,J],I,J)
        
    

In [9]:
def remove_multiple_nodes(data, delta, alpha, I=None, J = None):
    it = 1
    if I is None:
        I = np.arange(len(data))
    
    if J is None:    
        J = np.arange(len(data[0]))
        
    removal_ocurred = True
    
    while True:
        it += 1
        
        msr = MSR(data[I][:,J]) 
        
        if msr.H <delta:
            break
        if len(I) == 1 or len(J) == 1:
            break
            
        row_idxs_to_remove = np.nonzero(msr.HiJ > alpha*msr.H)[0]
        
        if len(row_idxs_to_remove) > 0:
            I = np.delete(I, row_idxs_to_remove)
        else:
            removal_ocurred = False
            
        msr = MSR(data[I][:,J])
        
        cols_idxs_to_remove= np.nonzero(msr.HIj > alpha*msr.H)[0]
        
        if len(cols_idxs_to_remove) > 0:
            J = np.delete(J, cols_idxs_to_remove)
            
        else:
            if not removal_ocurred:
                return remove_unique_nodes (data[I][:,J], delta, I, J)
    
    return (data[I][:,J],I,J)
        
        

In [11]:
def add_nodes(data, orig_data, alpha, I, J):
    it = 1
    orig_n, orig_m = orig_data.shape
    orig_I = np.arange(orig_n)
    orig_J = np.arange(orig_m)
    I_not_in_bicluster = np.setdiff1d(orig_I, I)
    J_not_in_bicluster = np.setdiff1d(orig_J, J)
    addition_ocurred = True
    while addition_ocurred:
        print ("%s iteration of node addition..." % it)
        it += 1
        addition_ocurred = False
        msr = MSR(orig_data[I][:,J])
        # column addition
        for j in J_not_in_bicluster:
            data_with_j = np.column_stack((data, orig_data[I][:,j]))
            msr_with_j = MSR(data_with_j)
            if msr_with_j.HIj[-1] <= msr.H:
                J = np.append(J, j)
                J_not_in_bicluster = np.delete(J_not_in_bicluster, np.where(j == J_not_in_bicluster)[0])
                data = data_with_j
                addition_ocurred = True
        msr = MSR(orig_data[I][:,J])
        # row addition
        for i in I_not_in_bicluster:
            data_with_i = np.row_stack((data, orig_data[i,J]))
            msr_with_i = MSR(data_with_i)
            if msr_with_i.HIj[-1] <= msr.H:
                I = np.append(I, i)
                I_not_in_bicluster = np.delete(I_not_in_bicluster, np.where(i == I_not_in_bicluster)[0])
                data = data_with_i
                adition_ocurred = True
# My guess is that we don't need invertions, because we are not in genes expression domain        
        # inverted row addition
#         msr = MSR(orig_data)
#         rows_idxs_to_add_overlap = np.nonzero(msr.HIj <= alpha*msr.H)[0]
#         rows_idxs_to_add = np.setdiff1d(rows_idxs_to_add_overlap, new_I)
#         if len(rows_idxs_to_add) > 0:
#             new_I = np.append(new_I, rows_idxs_to_add)
#             inverted_rows = np.fliplr(orig_data[rows_idxs_to_add][:,J])
#             data = np.row_stack((data, inverted_rows))
#         else:
#             addition_ocurred = False
    return (orig_data[I][:,J],I,J)


In [12]:
# %debug
def find_biclusters(data, delta, alpha, num_biclusters, num_iterations=1):
#     import pdb; pdb.set_trace()
    biclusters = []

    min_v = np.min(data)
    max_v = np.max(data)

    for i in range(num_biclusters):
        n, m = data.shape
        print data.shape

        plt.matshow(data, cmap=plt.cm.Blues, vmin=min_value, vmax=max_value)
        plt.show()

        print "--- Looking for bicluster %s:" % (i+1)

        # do not perform multiple node deletion
        if len(data) < 100 or len(data[0]) < 100:
            (C,I,J) = remove_unique_nodes(data, delta)
        else:
            (C,I,J) = remove_multiple_nodes(data, delta, alpha)

        (D,I,J) = add_nodes(C, data, alpha, I, J)

        biclusters.append(D.copy())

        plt.matshow(D, cmap=plt.cm.Blues, vmin=min_v, vmax=max_v)
        plt.show()

        A_line = mask(data, I, J, min_v, max_v)

    return biclusters

num_biclusters = 2
biclusters = find_biclusters(data, 1.0, 1.0, num_biclusters)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(print data.shape)? (<ipython-input-12-ef2246081bbd>, line 11)